In [ ]:
# Classe RSA que deve ter a geração de chaves a funcionar

class Rsa:
    def __init__(self, segParam):
        # inp = input('Parâmetro de Segurança: ')
        # segParam = int(inp)

        # https://www.inf.pucrs.br/~calazans/graduate/TPVLSI_I/RSA-oaep_spec.pdf --> Pág.15
        # https://stackoverflow.com/questions/34140589/how-to-get-a-prime-of-a-given-length-in-sage --> Informação random_prime

        self.modSize = segParam
        
        # COLOCAR NAS FUNÇÕES AUXILIARES
        def primerandom(l):
            return random_prime(2**l - 1, None, 2**(l-1))

        # random_prime(2**l - 1, None, 2**(l-1))  --> l = segParam/2
        # random_prime((2**(l//2)) - 1, None, (2**((l-1)//2)) + 1) --> l = segParam

        p = primerandom(modSize/2)
        q = primerandom(modSize/2)
        n = p*q

        phi = euler_phi(int(n))  # Euler totient function ou Carmichael's totient function

        # gcd(e, phi) = 1
        e = randrange(phi)
        while gcd(e,phi) != 1:
            e = randrange(phi)

        d = inverse_mod(e, phi)       # expoente da chave privada

        # Public Key: (n,e)
        self.publicKey = (n,e)
        # Private Key: (n,d)
        self.privateKey = (n,d)


In [ ]:
# Algumas funções auxiliares (as últimas duas é suposto fazerem o mesmo mas uma melhor que a outra)
import hashlib

def randomGen(pubKey):
    return randrange(pubKey[0] - 1)

# x-> inteiro a converter; l-> comprimento final
def i2osp(x, l):
    return int(x).to_bytes(l, 'big')
        
def os2ip(x):
    return int.from_bytes(x, 'big')

def byte_xor(ba1, ba2):
    return bytes([_a ^ _b for _a, _b in zip(ba1, ba2)])

def bxor(b1, b2): # use xor for bytes
    parts = []
    for b1, b2 in zip(b1, b2):
        parts.append(bytes([b1 ^ b2]))
    return b''.join(parts)

In [ ]:
def rsaEncrypt(self, k,  m):  # talvez seja necessário adicionar a chave pública aos argumentos
    return pow(m, k[1], k[0])

#rsaEncrypt((10142006496910083581, 5148364601967177589), 123131313)

def rsaDecrypt(self, k, c):
    return pow(c, k[1], k[0])

In [ ]:
# Esta era suposto ser a função do padding OAEP simples mas não funcionou por isso tentei fazer a de baixo
import hashlib

def byte_xor(ba1, ba2):
    return bytes([_a ^ _b for _a, _b in zip(ba1, ba2)])

def bxor(b1, b2): # use xor for bytes
    parts = []
    for b1, b2 in zip(b1, b2):
        parts.append(bytes([b1 ^ b2]))
    return b''.join(parts)

# Assume-se que a mensagem vem em bytes
def padding(n, msg):
    emTam = n//8
    
    m = msg
    hs = hashlib.sha256
    
    hTam = hs().digest_size
    
    mTam = len(m)
    
    rest = emTam - hTam
    
    if mTam > emTam - hTam: # ver se é preciso subtrair alguma coisa
        raise ValueError('Mensagem damasiado longa')
        exit()
    
    nZeros = hTam - mTam
    
    #k1 = b''
    #if rest > 0:
    k1 = b'\0' * nZeros
    
    r = os.urandom(hTam)
    
    #h.update(r)
    #rHash = h.digest()
    rHash = hs(r).digest()
    
    #mBytes = m.encode()  #str(m).to_bytes(mTam, 'big')
    
    #mBytes = base64.b64encode(m.encode("utf-8"))
    
    #print(mBytes)
    #mzeros = b"".join([m, k1])  # >>> b"".join([a, b])
    
    mzeros = m + k1
    
    #print(mzeros)
    
    eee = int.from_bytes(m, 'big')
    
    #show = (eee).to_bytes()
    print(eee)
    #print(mzeros)
    x = bxor(m, k1)
    
    #hHash = hashlib.sha256()
    #hHash.update(x)
    
    #h = hHash.digest()
    
    h = hs(x).digest()
    
    y = bxor(r, h)
    
    res = x + y
    
    
    
    return res

result = padding(512, b'asdadadas')

In [ ]:
# Aqui está uma das tentativas do encapsulamento e mais umas funções auxiliares relativas porque não consigo ler as de cima

import hashlib

def i2osp(x, l):
    return int(x).to_bytes(l, 'big')

def byte_xor(ba1, ba2):
    return bytes([_a ^ _b for _a, _b in zip(ba1, ba2)])

def bxor(b1, b2): # use xor for bytes
    parts = []
    for b1, b2 in zip(b1, b2):
        parts.append(bytes([b1 ^ b2]))
    return b''.join(parts)

def mgf(seed, l):
    
    hs = hashlib.sha256()
    hTam = hs.digest_size
    
    if l > 2**32 * hTam:
        print('Máscara demasiado grande')
        exit()
    
    t = b''
    for i in range (0, round(l/hTam) - 1): # ver o que fazer acerca da divisão
        c = i2osp(i, 4)
        t = t + hashlib.sha1(seed + c).digest()
        
    return t[:l]
    

def padding(msg, tam): # não esquecer o self nos argumentos; mensagem vem em bytes
    hs = hashlib.sha256
    p = b''
    
    m = msg
        
    emTam = tam//8
    #emTam = self.segParam//8
    mTam = len(m)
    hTam = hs().digest_size
    
    print('mTam: ' +str(mTam))
    print('emTam: ' +str(emTam))
    print('hTam :' +str(hTam))
    
    if mTam > (emTam - 2*hTam - 1): # Talvez "-2"
        raise ValueError('Erro: Mensagem demasiado grande')
        exit()
    
    pHash = hs(p).digest()
    
    ps = b'\0' * (emTam - mTam - 2*hTam - 1) # Talvez "-2"    
    
    #hs.update(p)
    #pHash = hs.digest()
    #pHash = hs(p).digest()
        
    # data block
    db = pHash + ps + b'\x01' + m
    
    print('....')
    
    print(pHash)
    print('PS :' + str(ps))
    print('m :' + str(m))
    
    print('....')
    
    # Gerar seed
    seed = os.urandom(hTam)
    
    dbMask = mgf(seed, (emTam - hTam - 1)) #ver se é preciso pôr "-1"
    
    print('db: ' + str(db))
    print('dbMask' + str(dbMask))
    
    maskedDB = bxor(db, dbMask)
    
    seedMask = mgf(maskedDB, hTam)
    
    maskedSeed = bxor(seed, seedMask)
    
    em = maskedSeed + maskedDB
    
    return em
    
#padding(b'xdcfvgbhnjmk,ctfyvgufyvgubhinjldfc ghjklgdtcfyvgubhinjodtfy', 1024)